In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import boto3
import glob
import io
from math import sqrt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from PIL import Image
import contextlib2
import dataset_util
import tf_record_creation_util
from collections import namedtuple, OrderedDict
from skimage.measure import regionprops
from sklearn.utils import shuffle

/home/ubuntu/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df = pd.read_csv("all/train_ship_segmentations.csv", header=0)
df_unique = df.drop_duplicates("ImageId")

In [3]:
df_random = df_unique.sample(frac=1)
th = np.random.rand(len(df_random)) < 0.9

In [4]:
train_test = df_random[th]

In [5]:
validation = df_random[~th]
train, test = train_test_split(train_test, test_size=0.2)

##### Create tf records 

In [7]:
def rle_decode(mask_rle, shape=(768, 768)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T  # Needed to align to RLE direction


def load_image(addr):
    # read an image and resize to (224, 224)
    # cv2 load images as BGR, convert it to RGB
    img = cv2.imread(addr)
#     print(img.shape)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.astype(np.float32)
    return img

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def rle_to_box(mask_rle):
    m = rle_decode(mask_rle)
    ps = regionprops(m)
    return ps[0].bbox

def create_tf_example(path, group):
    #with tf.gfile.GFile("/Users/yannis/Developpement/kaggle/airbus/all/train/"+path, 'rb') as fid:
        #encoded_jpg = fid.read()
        
    with tf.gfile.GFile("/home/ubuntu/kaggle-airbus_ship-faster-RCNN/all/train/"+path, 'rb') as fid:
        encoded_jpg = fid.read()
    
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)
    width, height = image.size

    filename = image.filename.encode('utf8')
    
#     filename = path.encode('utf8')
    image_format = b'jpg'
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes = []
    
    for g in group:
        if str(g) != "nan" : 
            pixel = rle_to_box(g)
            xmins.append(pixel[0] / width)
            ymins.append(pixel[1] / height)
            xmaxs.append(pixel[2] / width)
            ymaxs.append(pixel[3] / height)
            classes_text.append("ship".encode('utf8'))
            classes.append(1)
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/filename': dataset_util.bytes_feature(filename),
      'image/source_id': dataset_util.bytes_feature(filename),
#       'image/encoded':encoded_jpg,
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
      'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def create_record(dataset, name, df):
#     writer = tf.python_io.TFRecordWriter("datas/"+name+"-v2.records")
    
#     i = 0
#     for index, row in dataset.iterrows():
#         row = row.tolist()
#         group = df.loc[df['ImageId'] == row[0]]["EncodedPixels"]
#         tf_example = create_tf_example(row[0], group)
        
#         writer.write(tf_example.SerializeToString())
#         if i == 50:
#             break
#         i += 1

#     writer.close()
    
    num_shards=10
    output_filebase='datas/'+name+"/"+name+".records"

    with contextlib2.ExitStack() as tf_record_close_stack:
        output_tfrecords = tf_record_creation_util.open_sharded_output_tfrecords(tf_record_close_stack, output_filebase, num_shards)
        i=0
        for index, row in dataset.iterrows():
            row = row.tolist()
            group = df.loc[df['ImageId'] == row[0]]["EncodedPixels"]
            tf_example = create_tf_example(row[0], group)
            output_shard_index = i % num_shards
            output_tfrecords[output_shard_index].write(tf_example.SerializeToString())
            i += 1

Create train record

In [8]:
create_record(train, "train", df)

In [ ]:
create_record(test, "test", df)

In [ ]:
# bash
# aws s3 cp data/train.recods s3://kaggle-yannis/airbus/train.records
# aws s3 cp data/test.recods s3://kaggle-yannis/airbus/test.records

### Create TF record for resnet - tf

In [3]:
def create_tf_example(path):
        
    with tf.gfile.GFile("/home/ubuntu/kaggle/kaggle-airbus_ship-faster-RCNN/"+path, 'rb') as fid:
        encoded_jpg = fid.read()
    
    width, height = (350,350 )
    image_format = b'jpg'
    
    if "noship" in path:
        classes = [2]
    else:
        classes = [1]
    
    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height': dataset_util.int64_feature(height),
      'image/width': dataset_util.int64_feature(width),
      'image/encoded': dataset_util.bytes_feature(encoded_jpg),
      'image/format': dataset_util.bytes_feature(image_format),
      'image/class/label': dataset_util.int64_list_feature(classes),
    }))
    return tf_example


def create_record(dataset):
    writer = tf.python_io.TFRecordWriter("all/datas/train_classifier.tfrecord")
    
    i = 0
    for x in dataset:
        tf_example = create_tf_example(x)
        writer.write(tf_example.SerializeToString())

    writer.close()

In [4]:
create_record( shuffle(glob.glob("all/resize/350/*/*")) )